<a href="https://colab.research.google.com/github/fjpa121197/DataScienceDM/blob/master/Lab4/Lab4_bank_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>